<a href="https://colab.research.google.com/github/lx-jdar/progra-concurrente/blob/development/tp3/go/tp3_golang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo práctico 1: GOLANG

Instalacion de golang en Colab

In [ ]:
!add-apt-repository ppa:longsleep/golang-backports -y
!apt update
!apt install golang-go
%env GOPATH=/root/go
!go version

Creación del Archivo GO

In [15]:
%%writefile tp3-golang.go
package main

import (
	"fmt"
	"os"
	"strings"
	"sync"
)

const (
	LETRA_A      = 65
	CANT_THREADS = 2
)

var password []int
var mtx sync.Mutex

func convertirAEntero(cad string, start int, end int, wg *sync.WaitGroup) {
	defer wg.Done()
	mtx.Lock()
	for i := start; i < end; i++ {
		password[i] = int(cad[i]) - LETRA_A + 1
		// fmt.Printf("Char at %d Index Pos = %c int %d \n", i, cad[i], int(cad[i]))
	}
	mtx.Unlock()
}

func main() {
  if len(os.Args) < 1 {
    fmt.Println("Please, enter the parameter to convert!")
  }
	cadena := os.Args[1]
	password = make([]int, len(cadena))

	var wg sync.WaitGroup
	wg.Add(CANT_THREADS)
	fmt.Println("Analyzing Parte 1!")
	go convertirAEntero(strings.ToUpper(cadena), 0, 4, &wg)
	fmt.Println("Analyzing Parte 2!")
	go convertirAEntero(strings.ToUpper(cadena), 4, len(cadena), &wg)
	wg.Wait()

	fmt.Println("\nHello World!")
	fmt.Println(password)
}


Overwriting tp3-golang.go


Compilacion del Archivo GO

In [16]:
!go build tp3-golang.go

Ejecución del Programa

In [19]:
!./tp3-golang ARBoLiyOs

Analyzing Parte 1!
Analyzing Parte 2!

Hello World!
[1 18 2 15 12 9 25 15 19]
